# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [21]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import numpy as np
import pandas as pd
import pickle
import re

from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///etl.db')
df = pd.read_sql_table('etl.db', engine)
X = df['message']
Y = df[df.columns[-36:]]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):        
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    
    for tok in tokens:
        clean_token = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_token)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [26]:
y_pred = pipeline.predict(X_test)

def report_classification(y_test, y_pred):
    for idx, col in enumerate(y_test):
        set_y_pair = (y_test[col], y_pred[:, idx])
        avg='weighted'
        rep_col = "{}\n\tPrecision: {:.2f}%\n\tRecall: {:.2f}%\n\tF1 Score: {:.2f}%\n".format(col,
                                                                                 precision_score(*set_y_pair, average=avg), 
                                                                                 recall_score(*set_y_pair, average=avg), 
                                                                                 f1_score(*set_y_pair, average=avg))
        print(rep_col)
    print('Accuracy Score: {:.2f}%'.format(np.mean(y_test.values == y_pred)))

report_classification(y_test, y_pred)

related
	Precision: 0.66%
	Recall: 0.74%
	F1 Score: 0.68%

request
	Precision: 0.77%
	Recall: 0.83%
	F1 Score: 0.78%

offer
	Precision: 0.99%
	Recall: 1.00%
	F1 Score: 0.99%

aid_related
	Precision: 0.53%
	Recall: 0.57%
	F1 Score: 0.52%

medical_help
	Precision: 0.85%
	Recall: 0.91%
	F1 Score: 0.88%

medical_products
	Precision: 0.90%
	Recall: 0.94%
	F1 Score: 0.92%

search_and_rescue
	Precision: 0.95%
	Recall: 0.97%
	F1 Score: 0.96%

security
	Precision: 0.96%
	Recall: 0.98%
	F1 Score: 0.97%

military
	Precision: 0.95%
	Recall: 0.97%
	F1 Score: 0.95%

child_alone
	Precision: 1.00%
	Recall: 1.00%
	F1 Score: 1.00%

water
	Precision: 0.89%
	Recall: 0.94%
	F1 Score: 0.91%

food
	Precision: 0.82%
	Recall: 0.89%
	F1 Score: 0.84%

shelter
	Precision: 0.84%
	Recall: 0.91%
	F1 Score: 0.87%

clothing
	Precision: 0.97%
	Recall: 0.98%
	F1 Score: 0.98%

money
	Precision: 0.97%
	Recall: 0.98%
	F1 Score: 0.97%

missing_people
	Precision: 0.98%
	Recall: 0.99%
	F1 Score: 0.98%

refugees
	Precision: 0.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


hospitals
	Precision: 0.98%
	Recall: 0.99%
	F1 Score: 0.99%

shops
	Precision: 0.99%
	Recall: 1.00%
	F1 Score: 0.99%

aid_centers
	Precision: 0.98%
	Recall: 0.99%
	F1 Score: 0.98%

other_infrastructure
	Precision: 0.92%
	Recall: 0.96%
	F1 Score: 0.94%

weather_related
	Precision: 0.70%
	Recall: 0.73%
	F1 Score: 0.67%

floods
	Precision: 0.85%
	Recall: 0.91%
	F1 Score: 0.87%

storm
	Precision: 0.86%
	Recall: 0.91%
	F1 Score: 0.87%

fire
	Precision: 0.98%
	Recall: 0.99%
	F1 Score: 0.98%

earthquake
	Precision: 0.90%
	Recall: 0.91%
	F1 Score: 0.89%

cold
	Precision: 0.96%
	Recall: 0.98%
	F1 Score: 0.97%

other_weather
	Precision: 0.91%
	Recall: 0.95%
	F1 Score: 0.93%

direct_report
	Precision: 0.75%
	Recall: 0.81%
	F1 Score: 0.75%

Accuracy Score: 0.92%


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f1dd99ea620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [9]:
parameters = {
    'tfidf__use_idf': [True, False],
    #'clf__n_jobs': [1, 10, 50],
    #'clf__estimator__n_estimators': [10, 50, 90],
}

cv = GridSearchCV(pipeline, parameters)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [27]:
y_pred = cv.predict(X_test)

report_classification(y_test, y_pred)

related
	Precision: 0.67%
	Recall: 0.74%
	F1 Score: 0.68%

request
	Precision: 0.77%
	Recall: 0.83%
	F1 Score: 0.77%

offer
	Precision: 0.99%
	Recall: 1.00%
	F1 Score: 0.99%



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


aid_related
	Precision: 0.54%
	Recall: 0.57%
	F1 Score: 0.52%

medical_help
	Precision: 0.84%
	Recall: 0.92%
	F1 Score: 0.88%

medical_products
	Precision: 0.90%
	Recall: 0.95%
	F1 Score: 0.92%

search_and_rescue
	Precision: 0.94%
	Recall: 0.97%
	F1 Score: 0.96%

security
	Precision: 0.96%
	Recall: 0.98%
	F1 Score: 0.97%

military
	Precision: 0.93%
	Recall: 0.97%
	F1 Score: 0.95%

child_alone
	Precision: 1.00%
	Recall: 1.00%
	F1 Score: 1.00%

water
	Precision: 0.89%
	Recall: 0.94%
	F1 Score: 0.91%

food
	Precision: 0.82%
	Recall: 0.89%
	F1 Score: 0.84%

shelter
	Precision: 0.84%
	Recall: 0.91%
	F1 Score: 0.87%

clothing
	Precision: 0.97%
	Recall: 0.98%
	F1 Score: 0.98%

money
	Precision: 0.95%
	Recall: 0.98%
	F1 Score: 0.97%

missing_people
	Precision: 0.98%
	Recall: 0.99%
	F1 Score: 0.98%

refugees
	Precision: 0.94%
	Recall: 0.96%
	F1 Score: 0.95%

death
	Precision: 0.91%
	Recall: 0.95%
	F1 Score: 0.93%

other_aid
	Precision: 0.77%
	Recall: 0.86%
	F1 Score: 0.81%

infrastructure_relat

In [12]:
cv.best_params_

{'tfidf__use_idf': False}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
pipeline_2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier())),
])

In [14]:
pipeline_2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f1dd99ea620>, vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip_accents=None, token_pattern='(?u)\\

In [15]:
parameters_2 = {
    'clf__estimator__learning_rate': [0.5, 1.0, 1.5],
    'clf__estimator__n_estimators': [100, 200]
}

cv_2 = GridSearchCV(pipeline_2, parameters_2)
cv_2

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__learning_rate': [0.5, 1.0, 1.5], 'clf__estimator__n_estimators': [100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [16]:
cv_2.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__learning_rate': [0.5, 1.0, 1.5], 'clf__estimator__n_estimators': [100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [28]:
y_pred = cv_2.predict(X_test)

report_classification(y_test, y_pred)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


related
	Precision: 0.58%
	Recall: 0.76%
	F1 Score: 0.66%

request
	Precision: 0.78%
	Recall: 0.83%
	F1 Score: 0.78%

offer
	Precision: 0.99%
	Recall: 1.00%
	F1 Score: 0.99%

aid_related
	Precision: 0.54%
	Recall: 0.58%
	F1 Score: 0.49%

medical_help
	Precision: 0.86%
	Recall: 0.92%
	F1 Score: 0.88%

medical_products
	Precision: 0.90%
	Recall: 0.95%
	F1 Score: 0.92%

search_and_rescue
	Precision: 0.94%
	Recall: 0.97%
	F1 Score: 0.96%

security
	Precision: 0.96%
	Recall: 0.98%
	F1 Score: 0.97%

military
	Precision: 0.93%
	Recall: 0.96%
	F1 Score: 0.95%

child_alone
	Precision: 1.00%
	Recall: 1.00%
	F1 Score: 1.00%

water
	Precision: 0.88%
	Recall: 0.94%
	F1 Score: 0.91%

food
	Precision: 0.83%
	Recall: 0.89%
	F1 Score: 0.84%

shelter
	Precision: 0.84%
	Recall: 0.91%
	F1 Score: 0.87%

clothing
	Precision: 0.97%
	Recall: 0.98%
	F1 Score: 0.98%

money
	Precision: 0.95%
	Recall: 0.98%
	F1 Score: 0.97%

missing_people
	Precision: 0.98%
	Recall: 0.99%
	F1 Score: 0.98%

refugees
	Precision: 0.

In [18]:
cv_2.best_params_

{'clf__estimator__learning_rate': 0.5, 'clf__estimator__n_estimators': 100}

### 9. Export your model as a pickle file

In [19]:
with open('model.pkl', 'wb') as f:
    pickle.dump(cv2, f)

NameError: name 'cv2' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
%%writefile train_classifier.py

import sys
import nltk
#nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import pandas as pd
import pickle
import re

from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier


def load_data(database_filepath):
    engine = create_engine('sqlite:///' + database_filepath)
    
    df = pd.read_sql_table(database_filepath, engine)
    
    X = df['message']
    y = df[df.columns[-36:]]
    
    return X, y, df.columns[-36:]


def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    
    for tok in tokens:
        clean_token = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_token)
        
    return clean_tokens


def build_model():
    model = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators=100, 
                                                     learning_rate=0.5)))
    ])
    
    parameters = {
    'clf__estimator__learning_rate': [0.1, 0.2, 0.5],
    'clf__estimator__n_estimators': [100, 200, 300]
    }

    cv = GridSearchCV(model, parameters)
    
    return cv


def evaluate_model(model, X_test, Y_test, category_names):
    y_pred = model.predict(X_test)
    
    for idx, col in enumerate(category_names):
        set_y_pair = (y_test[col], y_pred[:, idx])
        avg='weighted'
        rep_col = "{}\n\tPrecision: {:.2f}%\n\tRecall: {:.2f}%\n\tF1 Score: {:.2f}%\n\tAccuracy: {:.2f}%".format(col,
                                                                                 precision_score(*set_y_pair, average=avg), 
                                                                                 recall_score(*set_y_pair, average=avg), 
                                                                                 f1_score(*set_y_pair, average=avg))
    print(rep_col)
    print('Accuracy Score: {}'.format(np.mean(Y_test.values == y_pred)))


def save_model(model, model_filepath):
    with open(model_filepath, 'wb') as f:
        pickle.dump(model, f)


def main():
    if len(sys.argv) == 3:
        database_filepath, model_filepath = sys.argv[1:]
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, Y, category_names = load_data(database_filepath)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
        
        print('Building model...')
        model = build_model()
        
        print('Training model...')
        model.fit(X_train, Y_train)
        
        print('Evaluating model...')
        evaluate_model(model, X_test, Y_test, category_names)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')

    else:
        print('Please provide the filepath of the disaster messages database '\
              'as the first argument and the filepath of the pickle file to '\
              'save the model to as the second argument. \n\nExample: python '\
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()